In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
import statistics as stats
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor


In [3]:
df = pd.read_csv('train.csv')

In [4]:
df.head()

,Year,Month,ProductCategory,Sales(In ThousandDollars)
0,2009,1,WomenClothing,1755.0
1,2009,1,MenClothing,524.0
2,2009,1,OtherClothing,936.0
3,2009,2,WomenClothing,1729.0
4,2009,2,MenClothing,496.0


In [13]:
#read excel file with multiple sheets
file = pd.read_excel('macro_economic.xlsx')
file

,Year-Month,Monthly Nominal GDP Index (inMillion$),Monthly Real GDP Index (inMillion$),CPI,PartyInPower,unemployment rate,CommercialBankInterestRateonCreditCardPlans,"Finance Rate on Personal Loans at Commercial Banks, 24 Month Loan",Earnings or wages in dollars per hour,AdvertisingExpenses (in Thousand Dollars),Cotton Monthly Price - US cents per Pound(lbs),Change(in%),Average upland planted(million acres),Average upland harvested(million acres),yieldperharvested acre,Production (in 480-lb netweright in million bales),Mill use (in 480-lb netweright in million bales),Exports
0,2009 - Jan,14421.752895,14407.053343,233.402,Democrats,7.8,12.03,11.44,22.05,137,57.70,4.02,9.296,7.559,799,12.589,4.170,11.550
1,2009 - Feb,14389.200466,14366.176571,234.663,Democrats,8.3,12.97,11.05,22.22,200,55.21,-4.32,9.296,7.559,799,12.589,3.870,11.100
2,2009 - Mar,14340.701639,14351.786822,235.067,Democrats,8.7,12.97,11.05,22.22,?,51.50,-6.72,9.296,7.559,799,12.589,3.720,11.650
3,2009 - Apr,14326.815525,14351.601731,235.582,Democrats,9.0,12.97,11.05,22.13,214,56.78,10.25,9.296,7.559,787,12.400,3.620,12.225
4,2009 - May,14345.904809,14368.123959,235.975,Democrats,9.4,13.32,11.25,22.04,?,61.95,9.11,9.297,7.400,803,12.384,3.520,12.300
5,2009 - Jun,14348.530666,14346.820106,237.172,Democrats,9.5,13.32,11.25,21.96,?,61.39,-0.90,8.668,7.760,796,12.876,3.470,10.300
6,2009 - Jul,14317.372922,14345.676097,237.600,Democrats,9.5,13.32,11.25,22.00,?,64.80,5.55,8.905,7.753,796,12.863,3.470,9.700
7,2009 - Aug,14424.284901,14435.812509,238.282,Democrats,9.6,13.71,10.89,22.16,?,64.26,-0.83,8.905,7.619,809,12.840,3.470,9.700
8,2009 - Sep,14410.777177,14425.972752,238.568,Democrats,9.8,13.71,10.89,22.20,?,64.07,-0.30,8.989,7.582,827,13.071,3.470,10.000
9,2009 - Oct,14596.605447,14595.393094,238.380,Democrats,10.0,13.71,10.89,22.27,?,66.82,4.29,8.989,7.586,799,12.631,3.370,10.000


In [10]:
# checking shape of each sheet
for i in file.sheet_names:
    data = pd.read_excel(file, sheet_name= i )
    print(data.shape)


(96, 18)


In [15]:
# read weather data excel sheet into df_weather
df_macro = pd.read_excel('macro_economic.xlsx', sheetname=None, ignore_index=True)
df_macro

C:\Users\ASHISH\Anaconda3\lib\site-packages\pandas\io\excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


OrderedDict([('Sheet1',
                  Year-Month  Monthly Nominal GDP Index (inMillion$)  \
              0   2009 - Jan                            14421.752895   
              1   2009 - Feb                            14389.200466   
              2   2009 - Mar                            14340.701639   
              3   2009 - Apr                            14326.815525   
              4   2009 - May                            14345.904809   
              5   2009 - Jun                            14348.530666   
              6   2009 - Jul                            14317.372922   
              7   2009 - Aug                            14424.284901   
              8   2009 - Sep                            14410.777177   
              9   2009 - Oct                            14596.605447   
              10  2009 - Nov                            14590.432252   
              11  2009 - Dec                            14512.495301   
              12  2010 - Jan            

In [50]:
# concat all sheets into one
cdf_macro = pd.concat(df_macro.values())

cdf_macro.head()

,Year-Month,Monthly Nominal GDP Index (inMillion$),Monthly Real GDP Index (inMillion$),CPI,PartyInPower,unemployment rate,CommercialBankInterestRateonCreditCardPlans,"Finance Rate on Personal Loans at Commercial Banks, 24 Month Loan",Earnings or wages in dollars per hour,AdvertisingExpenses (in Thousand Dollars),Cotton Monthly Price - US cents per Pound(lbs),Change(in%),Average upland planted(million acres),Average upland harvested(million acres),yieldperharvested acre,Production (in 480-lb netweright in million bales),Mill use (in 480-lb netweright in million bales),Exports
0,2009 - Jan,14421.752895,14407.053343,233.402,Democrats,7.8,12.03,11.44,22.05,137,57.70,4.02,9.296,7.559,799,12.589,4.17,11.550
1,2009 - Feb,14389.200466,14366.176571,234.663,Democrats,8.3,12.97,11.05,22.22,200,55.21,-4.32,9.296,7.559,799,12.589,3.87,11.100
2,2009 - Mar,14340.701639,14351.786822,235.067,Democrats,8.7,12.97,11.05,22.22,?,51.50,-6.72,9.296,7.559,799,12.589,3.72,11.650
3,2009 - Apr,14326.815525,14351.601731,235.582,Democrats,9.0,12.97,11.05,22.13,214,56.78,10.25,9.296,7.559,787,12.400,3.62,12.225
4,2009 - May,14345.904809,14368.123959,235.975,Democrats,9.4,13.32,11.25,22.04,?,61.95,9.11,9.297,7.400,803,12.384,3.52,12.300


In [17]:
df.rename(columns = {'Sales(In ThousandDollars)' : 'Sales'}, inplace= True)

In [18]:
df['ProductCategory'] = df['ProductCategory'].astype('category', categories = df['ProductCategory'].unique( ))

C:\Users\ASHISH\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  """Entry point for launching an IPython kernel.


In [19]:
df['ProductCategory'].replace(['OtherClothing','MenClothing','WomenClothing'] , [3,2,1], inplace = True)

In [20]:
df.isna().apply(pd.value_counts)

,Year,Month,ProductCategory,Sales
False,216.0,216.0,216.0,204
True,NaN,NaN,NaN,12


In [21]:
# simple loop to replace NaN values with mean of adjacent sales of same category
x = []
for i in range(0,len(df.Sales)):
    if df.isna().Sales[i] == True :
        x.append(stats.mean([df.Sales[i+3],df.Sales[i-3]]))
        
    else:
        x.append(df.Sales[i])
Sales = pd.Series(x)


In [22]:
df['Sales'] = Sales

In [23]:
df.isna().apply(pd.value_counts)

,Year,Month,ProductCategory,Sales
False,216,216,216,216


In [24]:
# Final sales data
df.head()

,Year,Month,ProductCategory,Sales
0,2009,1,1,1755.0
1,2009,1,2,524.0
2,2009,1,3,936.0
3,2009,2,1,1729.0
4,2009,2,2,496.0


In [25]:
# Final sales data
df.head()

,Year,Month,ProductCategory,Sales
0,2009,1,1,1755.0
1,2009,1,2,524.0
2,2009,1,3,936.0
3,2009,2,1,1729.0
4,2009,2,2,496.0


In [49]:
cdf_macro.head()

,Year-Month,Monthly Nominal GDP Index (inMillion$),Monthly Real GDP Index (inMillion$),CPI,PartyInPower,unemployment rate,CommercialBankInterestRateonCreditCardPlans,"Finance Rate on Personal Loans at Commercial Banks, 24 Month Loan",Earnings or wages in dollars per hour,Cotton Monthly Price - US cents per Pound(lbs),Change(in%),Average upland planted(million acres),Average upland harvested(million acres),yieldperharvested acre,Production (in 480-lb netweright in million bales),Mill use (in 480-lb netweright in million bales),Exports
0,2009 - Jan,14421.752895,14407.053343,233.402,Democrats,7.8,12.03,11.44,22.05,57.70,4.02,9.296,7.559,799,12.589,4.17,11.550
1,2009 - Feb,14389.200466,14366.176571,234.663,Democrats,8.3,12.97,11.05,22.22,55.21,-4.32,9.296,7.559,799,12.589,3.87,11.100
2,2009 - Mar,14340.701639,14351.786822,235.067,Democrats,8.7,12.97,11.05,22.22,51.50,-6.72,9.296,7.559,799,12.589,3.72,11.650
3,2009 - Apr,14326.815525,14351.601731,235.582,Democrats,9.0,12.97,11.05,22.13,56.78,10.25,9.296,7.559,787,12.400,3.62,12.225
4,2009 - May,14345.904809,14368.123959,235.975,Democrats,9.4,13.32,11.25,22.04,61.95,9.11,9.297,7.400,803,12.384,3.52,12.300


In [57]:
cdf_macro = data.drop(labels= "AdvertisingExpenses (in Thousand Dollars)" , axis = 1)

In [58]:
cdf_macro.fillna(value = 'Normal', inplace = True)

In [59]:
cdf_macro.replace(to_replace= '-', value= np.NaN , inplace= True)

In [60]:
cdf_macro.replace(to_replace= 'T', value= np.NaN , inplace= True)

In [61]:
cdf_macro.isna().apply(pd.value_counts)

,Year-Month,Monthly Nominal GDP Index (inMillion$),Monthly Real GDP Index (inMillion$),CPI,PartyInPower,unemployment rate,CommercialBankInterestRateonCreditCardPlans,"Finance Rate on Personal Loans at Commercial Banks, 24 Month Loan",Earnings or wages in dollars per hour,Cotton Monthly Price - US cents per Pound(lbs),Change(in%),Average upland planted(million acres),Average upland harvested(million acres),yieldperharvested acre,Production (in 480-lb netweright in million bales),Mill use (in 480-lb netweright in million bales),Exports
False,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96


In [46]:
cdf_macro = cdf_macro.interpolate()

In [47]:
cdf_macro.isna().apply(pd.value_counts)

,Year-Month,Monthly Nominal GDP Index (inMillion$),Monthly Real GDP Index (inMillion$),CPI,PartyInPower,unemployment rate,CommercialBankInterestRateonCreditCardPlans,"Finance Rate on Personal Loans at Commercial Banks, 24 Month Loan",Earnings or wages in dollars per hour,Cotton Monthly Price - US cents per Pound(lbs),Change(in%),Average upland planted(million acres),Average upland harvested(million acres),yieldperharvested acre,Production (in 480-lb netweright in million bales),Mill use (in 480-lb netweright in million bales),Exports
False,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96


In [54]:
cdf_macro.Date.replace( to_replace= ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec'] , value = [1,2,3,4,5,6,7,8,9,10,11,12], inplace = True)

In [51]:
cdf_macro = cdf_macro.rename(columns={'Year-Month': 'Date'})

In [55]:
cdf_macro.Date.replace( to_replace= ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec'] , value = [1,2,3,4,5,6,7,8,9,10,11,12], inplace = True)

In [62]:
cdf_macro

,Year-Month,Monthly Nominal GDP Index (inMillion$),Monthly Real GDP Index (inMillion$),CPI,PartyInPower,unemployment rate,CommercialBankInterestRateonCreditCardPlans,"Finance Rate on Personal Loans at Commercial Banks, 24 Month Loan",Earnings or wages in dollars per hour,Cotton Monthly Price - US cents per Pound(lbs),Change(in%),Average upland planted(million acres),Average upland harvested(million acres),yieldperharvested acre,Production (in 480-lb netweright in million bales),Mill use (in 480-lb netweright in million bales),Exports
0,2009 - Jan,14421.752895,14407.053343,233.402,Democrats,7.8,12.03,11.44,22.05,57.70,4.02,9.296,7.559,799,12.589,4.170,11.550
1,2009 - Feb,14389.200466,14366.176571,234.663,Democrats,8.3,12.97,11.05,22.22,55.21,-4.32,9.296,7.559,799,12.589,3.870,11.100
2,2009 - Mar,14340.701639,14351.786822,235.067,Democrats,8.7,12.97,11.05,22.22,51.50,-6.72,9.296,7.559,799,12.589,3.720,11.650
3,2009 - Apr,14326.815525,14351.601731,235.582,Democrats,9.0,12.97,11.05,22.13,56.78,10.25,9.296,7.559,787,12.400,3.620,12.225
4,2009 - May,14345.904809,14368.123959,235.975,Democrats,9.4,13.32,11.25,22.04,61.95,9.11,9.297,7.400,803,12.384,3.520,12.300
5,2009 - Jun,14348.530666,14346.820106,237.172,Democrats,9.5,13.32,11.25,21.96,61.39,-0.90,8.668,7.760,796,12.876,3.470,10.300
6,2009 - Jul,14317.372922,14345.676097,237.600,Democrats,9.5,13.32,11.25,22.00,64.80,5.55,8.905,7.753,796,12.863,3.470,9.700
7,2009 - Aug,14424.284901,14435.812509,238.282,Democrats,9.6,13.71,10.89,22.16,64.26,-0.83,8.905,7.619,809,12.840,3.470,9.700
8,2009 - Sep,14410.777177,14425.972752,238.568,Democrats,9.8,13.71,10.89,22.20,64.07,-0.30,8.989,7.582,827,13.071,3.470,10.000
9,2009 - Oct,14596.605447,14595.393094,238.380,Democrats,10.0,13.71,10.89,22.27,66.82,4.29,8.989,7.586,799,12.631,3.370,10.000
